### Import packages:

In [820]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

from collections import Counter
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

#Show all columns, rows, width
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Read & Inspect data:

In [821]:
train_df = pd.read_csv('./data/train.csv', index_col = 0)
test_df = pd.read_csv('./data/test.csv', index_col = 0)

In [822]:
# #log transformation:
# for col in ['GrLivArea','LotArea','LotFrontage', '1stFlrSF']:
#     train_df[col] = np.log(train_df[col])
#     test_df[col] = np.log(train_df[col])

## Combine data set:

In [823]:
combine = pd.concat([train_df, test_df])

## Data Cleaning:

### Missing imputation:

In [824]:
# Function impute_missing takes dataframe which needs to be imputed as input variable
def impute_missing(dfName):
    
 # impute GarageYrBlt with min GarageYrBlt for houses with no garage    
    #dfName['GarageYrBlt'] = dfName['GarageYrBlt'].fillna(min(dfName['GarageYrBlt']))
    #dfName['GarageYrBlt'] = dfName['GarageYrBlt'].fillna(dfName['YearBuilt']-50)
    dfName['GarageYrBlt'] = dfName['GarageYrBlt'].fillna(0)

 # impute MasVnrArea to 0 because for NAs we are imputing type to None   
    dfName['MasVnrArea'] = dfName['MasVnrArea'].fillna(0)
    dfName['BsmtFinSF1'] = dfName['BsmtFinSF1'].fillna(0)
    dfName['BsmtFinSF2'] = dfName['BsmtFinSF2'].fillna(0)
    dfName['BsmtUnfSF'] = dfName['BsmtUnfSF'].fillna(0)
    dfName['TotalBsmtSF'] = dfName['TotalBsmtSF'].fillna(0)
    dfName['BsmtFullBath'] = dfName['BsmtFullBath'].fillna(0)
    dfName['BsmtHalfBath'] = dfName['BsmtHalfBath'].fillna(0)
    dfName['GarageCars'] = dfName['GarageCars'].fillna(0)
    dfName['GarageArea'] = dfName['GarageArea'].fillna(0)
    dfName['SalePrice'] = dfName['SalePrice'].fillna(0)

 #impute missing values by the median LotFrontage of the neighborhood.
    dfName['LotFrontage'] = dfName.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
#     meanLA = dfName['LotArea'].mean()
#     meanLF = dfName['LotFrontage'].mean()
#     ratio = meanLF/meanLA
#     dfName['LotFrontage'] = dfName['LotFrontage'].fillna(ratio*dfName['LotArea'])
 
 # for missing in Electrical variable, impute with type which occurs most frequently
    dfName['Electrical'] = dfName['Electrical'].fillna(dfName['Electrical'].mode()[0]) 
    dfName['MSZoning'] = dfName['MSZoning'].fillna(dfName['MSZoning'].mode()[0]) 
    dfName['Utilities'] = dfName['Utilities'].fillna(dfName['Utilities'].mode()[0]) 
    dfName['Exterior1st'] = dfName['Exterior1st'].fillna(dfName['Exterior1st'].mode()[0]) 
    dfName['Exterior2nd'] = dfName['Exterior2nd'].fillna(dfName['Exterior2nd'].mode()[0]) 
    dfName['KitchenQual'] = dfName['KitchenQual'].fillna(dfName['KitchenQual'].mode()[0])
    dfName['Functional'] = dfName['Functional'].fillna(dfName['Functional'].mode()[0])
    dfName['KitchenQual'] = dfName['KitchenQual'].fillna(dfName['KitchenQual'].mode()[0])
    dfName['SaleType'] = dfName['SaleType'].fillna(dfName['SaleType'].mode()[0])

 # Check col names ad imputes None, 0 , or aggregated value based on algorithm       
    for col in ('PoolQC','MiscFeature','Alley','Fence','FireplaceQu','GarageType','GarageFinish', 'GarageQual', 'GarageCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1','BsmtFinType2', 'MasVnrType'):
        dfName[col] = dfName[col].fillna('None') 

In [825]:
impute_missing(combine)

## Dummify:

In [826]:
cat_features = combine.dtypes[combine.dtypes == 'object'].index
num_features = combine.dtypes[combine.dtypes != 'object'].index
cat_extra = ['MSSubClass', 'OverallCond', 'OverallQual']
combine[cat_extra] = combine[cat_extra].astype('object')
cat_features = combine.dtypes[combine.dtypes == 'object'].index
num_features = combine.dtypes[combine.dtypes != 'object'].index

In [827]:
# cont_col = ['LotFrontage', 'LotArea',  'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
#           'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
#           'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
#            'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
#            'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
#            'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
#            'MoSold', 'GarageYrBlt', 'YrSold', 'YearBuilt','YearRemodAdd']

# cat_col = list(set(train_df.columns)-set(cont_col)-{'SalePrice'})

# #define get dummy function for all dummy variables
# def get_dummy(df, cols):
#     for col in cols:
#         dummies = pd.get_dummies(df[col], prefix=col, dummy_na=False)
#         df = df.drop(col, 1)
#         df = pd.concat([df, dummies], axis=1)
#     return df

# combine_d = get_dummy(combine, cat_col)

# print(combine_d.shape)

# dtrain = combine_d[combine_d.SalePrice.notnull()]
# dtest = combine_d[combine_d.SalePrice.isnull()].drop(columns = 'SalePrice')

# print(dtrain.shape)
# print(dtest.shape)

In [828]:
combine = pd.get_dummies(combine, drop_first=True)
dtrain = combine[combine.SalePrice!=0]
dtest = combine[combine.SalePrice==0].drop('SalePrice',1)
print(dtrain.shape)
print(dtest.shape)
dtrain.sample(5)

(1460, 289)
(1459, 288)


,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,Fireplaces,FullBath,GarageArea,GarageCars,GarageYrBlt,GrLivArea,HalfBath,KitchenAbvGr,LotArea,LotFrontage,LowQualFinSF,MasVnrArea,MiscVal,MoSold,OpenPorchSF,PoolArea,SalePrice,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,Alley_None,Alley_Pave,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtCond_Gd,BsmtCond_None,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_None,BsmtFinType2_Rec,BsmtFinType2_Unf,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_TA,CentralAir_Y,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_None,FireplaceQu_Po,FireplaceQu_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageCond_Fa,GarageCond_Gd,GarageCond_None,GarageCond_Po,GarageCond_TA,GarageFinish_None,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_None,GarageQual_Po,GarageQual_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,LandContour_HLS,LandContour_Low,LandContour_Lvl,LandSlope_Mod,LandSlope_Sev,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LotShape_IR2,LotShape_IR3,LotShape_Reg,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,OverallCond_2,OverallCond_3,OverallCond_4,OverallCond_5,OverallCond_6,OverallCond_7,OverallCond_8,OverallCond_9,OverallQual_2,OverallQual_3,OverallQual_4,OverallQual_5,OverallQual_6,OverallQual_7,OverallQual_8,OverallQual_9,OverallQual_10,PavedDri

## Outlier:

In [829]:
dtrain = dtrain.drop(dtrain[(dtrain.GrLivArea>4000) & (dtrain.SalePrice<300000)].index)
dtrain.shape

(1458, 289)

## Feature Engineering:

#### Add categorical col for 0 value:

In [837]:
inputcols = ['2ndFlrSF','BsmtFinSF1','BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',\
             'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch',\
            'LowQualFinSF', 'PoolArea', 'MasVnrArea', 'MiscVal']
outputcols = ['has2ndflr', 'hasbsmt1', 'hasbsmt2', 'isbsmtcomplete', 'hasbsmt', 'hasgarage', 'haswooddeck', 'hasopenporch',\
             'hasenclosedporch', 'has3ssnporch', 'hasscreenporch', 'islowqualfin', 'haspool', 'hasmasvnr', 'hasmiscval']

def add_categorical_col(df, inputcols, outputcols):
    if len(inputcols) != len(outputcols):
        raise "Col len does not equal"
    for index,inputcol in enumerate(inputcols):
        df[outputcols[index]] = df[inputcol].apply(lambda x: 0 if x == 0 else 1)

add_categorical_col(dtrain, inputcols,outputcols)
add_categorical_col(dtest, inputcols,outputcols)
print(dtrain.shape)
print(dtest.shape)
dtrain.sample(6)

(1458, 322)
(1459, 321)


,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,Fireplaces,FullBath,GarageArea,GarageCars,GarageYrBlt,GrLivArea,HalfBath,KitchenAbvGr,LotArea,LotFrontage,LowQualFinSF,MasVnrArea,MiscVal,MoSold,OpenPorchSF,PoolArea,SalePrice,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,Alley_None,Alley_Pave,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtCond_Gd,BsmtCond_None,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_None,BsmtFinType2_Rec,BsmtFinType2_Unf,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_TA,CentralAir_Y,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_None,FireplaceQu_Po,FireplaceQu_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageCond_Fa,GarageCond_Gd,GarageCond_None,GarageCond_Po,GarageCond_TA,GarageFinish_None,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_None,GarageQual_Po,GarageQual_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,LandContour_HLS,LandContour_Low,LandContour_Lvl,LandSlope_Mod,LandSlope_Sev,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LotShape_IR2,LotShape_IR3,LotShape_Reg,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,OverallCond_2,OverallCond_3,OverallCond_4,OverallCond_5,OverallCond_6,OverallCond_7,OverallCond_8,OverallCond_9,OverallQual_2,OverallQual_3,OverallQual_4,OverallQual_5,OverallQual_6,OverallQual_7,OverallQual_8,OverallQual_9,OverallQual_10,PavedDri

#### Add ratio col:

In [831]:
def add_col_ratio(df):
    df['tota'] = df['LotArea'] + df['BsmtFinSF1'] + df['BsmtFinSF2'] + df['1stFlrSF'] + df['2ndFlrSF'] + df['GrLivArea'] + df['GarageArea']
    df['totfb'] = df['BsmtFullBath'] + df['FullBath']
    df['tothb'] = df['BsmtHalfBath'] + df['HalfBath']
    df['totbabgr'] = df['FullBath'] + df['HalfBath']
    df['totb'] = df['BsmtFullBath'] + df['BsmtHalfBath'] + df['BedroomAbvGr']
    
    df['tota_totb'] = df['totb']/df['tota']
    df['tota_totfb'] = df['totfb']/df['tota']
    df['tota_tothb'] = df['tothb']/df['tota']
    df['tota_br'] = df['BedroomAbvGr']/df['tota']
    df['tota_kc'] = df['KitchenAbvGr']/df['tota']
    df['tota_totrs'] = df['TotRmsAbvGrd']/df['tota']
    df['tota_gc'] = df['GarageCars']/df['tota']
    df['totbath_br'] = df['BedroomAbvGr']/(df['totbabgr']+1)
    df['totfb_br'] = df['totfb']/(df['BedroomAbvGr']+1)
    df['tothb_br'] = df['tothb']/(df['BedroomAbvGr']+1)
    df['totb_totrm'] = df['TotRmsAbvGrd']/(df['totb']+1)
    df['brm_kitchen'] = df['KitchenAbvGr']/(df['BedroomAbvGr']+1)
    df['totrm_garacar'] = df['GarageCars']/(df['TotRmsAbvGrd']+1)
    
add_col_ratio(dtrain)
add_col_ratio(dtest)
print(dtrain.shape)
print(dtest.shape)

(1458, 322)
(1459, 321)


## Feature selection:

In [832]:
# import sklearn.feature_selection as fs
# dtrain = fs.VarianceThreshold(threshold = 1).fit_transform(dtrain)
# dtest = fs.VarianceThreshold(threshold = 1).fit_transform(dtest)

# dtrain
# features = dtrain.drop('SalePrice', axis = 1)
# price = dtrain.SalePrice
# dtrain = fs.SelectKBest(fs.chi2, k=250).fit_transform(features, price)
# dtrain.shape
#dtrain = pd.DataFrame(dtrain)

## Pre-Modeling:

In [833]:
import sklearn.model_selection as ms
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error
from math import sqrt
import sklearn.metrics
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
lasso = Lasso()
ridge = Ridge()
net = ElasticNet()

In [834]:
# features = dtrain
features = dtrain.drop('SalePrice', axis = 1)
price = np.log(dtrain.SalePrice)

X_train, X_test, y_train, y_test = train_test_split(features, price, test_size=0.2, random_state=38)

param_grid = [
  {'alpha': list(np.linspace(1e-10,100,10))}
 ]

cv_lasso = GridSearchCV(lasso, param_grid, cv=10, n_jobs=-1)
cv_lasso.fit(X_train, y_train)
print(cv_lasso.best_params_)
print(cv_lasso.best_score_)
alpha_lasso = cv_lasso.best_params_['alpha']

cv_lasso.set_params()
cv_lasso.fit(X_train, y_train)
print('This is R^2 %f!' %cv_lasso.score(X_test, y_test))
ypred = cv_lasso.predict(X_test)
print('This is RMSE %f!' %sqrt(mean_squared_error(np.log(y_test), np.log(ypred))))


{'alpha': 1e-10}
0.8480294146477395
This is R^2 0.931090!
This is RMSE 0.009186!


In [835]:
#features = dtrain

features = dtrain.drop('SalePrice', axis = 1)
price = np.log(dtrain.SalePrice)

X_train, X_test, y_train, y_test = train_test_split(features, price, test_size=0.2, random_state=38)

param_grid = [
  {'alpha': list(np.linspace(1e-10,100,10))}
 ]

cv_ridge = GridSearchCV(ridge, param_grid, cv=10, n_jobs=-1)
cv_ridge.fit(features,price)
print(cv_ridge.best_params_)
print(cv_ridge.best_score_)
alpha_ridge = cv_ridge.best_params_['alpha']

cv_ridge.set_params()
cv_ridge.fit(X_train, y_train)
print('This is R^2 %f!' %cv_ridge.score(X_test, y_test))
ypred = cv_ridge.predict(X_test)
print('This is RMSE %f!' %sqrt(mean_squared_error(np.log(y_test), np.log(ypred))))

{'alpha': 11.1111111112}
0.915275172435269
This is R^2 0.939936!
This is RMSE 0.008589!


In [734]:
#Tree model
# from sklearn import tree
# tree_model = tree.DecisionTreeClassifier()

# features = dtrain.drop('SalePrice', axis = 1)
# price = np.log(dtrain.SalePrice)

# X_train, X_test, y_train, y_test = train_test_split(features, price, test_size=0.2, random_state=38)

# tree_model.fit(X_train, y_train)
# print('This is R^2 %f!' %tree_model.score(X_test, y_test))
# ypred = tree_model.predict(X_test)
# print('This is RMSE %f!' %sqrt(mean_squared_error(np.log(y_test), np.log(ypred))))

ValueError: Unknown label type: 'continuous'

## Modeling:

In [818]:
#Lasso
features = dtrain.drop('SalePrice', axis = 1)
price = np.log(dtrain.SalePrice)

param_grid = [
  {'alpha': list(np.linspace(1e-10,100,10))}
 ]

cv_lasso = GridSearchCV(lasso, param_grid, cv=10, n_jobs=-1)
cv_lasso.fit(features,price)
print(cv_lasso.best_params_)
print(cv_lasso.best_score_)
alpha_lasso = cv_lasso.best_params_['alpha']

cv_lasso.set_params()
cv_lasso.fit(features, price)
cv_lasso.score(features, price)

{'alpha': 1e-10}
0.8989943560348241


0.9533636028099233

In [819]:
# Ridge
features = dtrain.drop('SalePrice', axis = 1)
price = np.log(dtrain.SalePrice)

param_grid = [
  {'alpha': list(np.linspace(1e-10,100,10))}
 ]

cv_ridge = GridSearchCV(ridge, param_grid, cv=10, n_jobs=-1)
cv_ridge.fit(features,price)
print(cv_ridge.best_params_)
print(cv_ridge.best_score_)
alpha_ridge = cv_ridge.best_params_['alpha']

cv_ridge.set_params()
cv_ridge.fit(features, price)
cv_ridge.score(features, price)

{'alpha': 11.1111111112}
0.9152658696919629


0.9399044529837347

In [516]:

# features = dtrain.drop('SalePrice', axis = 1)
# price = np.log(dtrain.SalePrice)

# param_grid = [
#   {'alpha': list(np.linspace(1e-10,100,10)), 'l1_ratio': np.arange(0.0, 1.0, 0.1)}
#  ]

# enet = ElasticNet()
# cv_enet = GridSearchCV(enet, param_grid, cv=10, n_jobs=-1)
# cv_enet.fit(features,price)
# print(cv_enet.best_params_)
# print(cv_enet.best_score_)

# cv_enet.set_params()
# cv_enet.fit(features, price)
# cv_enet.score(features, price)

## Write Data:

In [517]:
x = pd.concat([dtest.reset_index(), pd.Series(np.exp(cv_lasso.predict(dtest))).rename('SalePrice')], \
              axis=1)[['Id', 'SalePrice']].set_index('Id')
x.to_csv('lasso_result.csv')

In [596]:
x = pd.concat([dtest.reset_index(), pd.Series(np.exp(cv_ridge.predict(dtest))).rename('SalePrice')], axis=1)[['Id', 'SalePrice']].set_index('Id')
x.to_csv('ridge_result.csv')

In [26]:
x = pd.concat([dtest.reset_index(), pd.Series(np.exp(cv_enet.predict(dtest))).rename('SalePrice')], axis=1)[['Id', 'SalePrice']].set_index('Id')
x.to_csv('enet_result.csv')